# RQ4
## Impact of M and w/ (w/o) selection

In [8]:
import pandas as pd

In [25]:
def sample_maxGen(df, maxGen=20):
    df_all = None
    for oid in range(0, max(df['OID'])+1):
        if oid not in df['OID']:
            print('skip: oid = ', oid)
            continue
        df1 = df[df.OID == oid]
        if len(set(df1.pairConsistent.values)) == 2:
            dff = df1[df1.pairConsistent == False]
            if len(dff) >= maxGen:
                df1 = dff.iloc[:maxGen, :]
            else:
                dft = df1[df1.pairConsistent == True].iloc[:maxGen - len(dff), :]
                df1 = pd.concat([dff, dft], axis=0)
        else:
            df1 = df[df.OID == oid].iloc[:min(len(df1), maxGen), :]

        if df_all is None:
            df_all = df1
        else:
            df_all = pd.concat([df_all, df1], axis=0)
            # print(len(df_all))
        # print()
    return df_all

def setup_groundtruth(cr='NeuralCoref', gt_method='Textattack'):  # Stats
    # Because Checklist has all 200 sentences recorded, so we use it as ground truth.
    df = pd.read_csv('../Output/RQ1-IssueRevealing/generate/{}/{}.tsv'.format(cr, gt_method), delimiter='\t')
    print(len(set(df['OID'])))
    
    for i in range(0, 1001):
        if i not in set(df['OID']):
            print(i)
    assert len(set(df['OID'])) == 1000

    Groundtruth_N = set(df[df['oriConsistent'] == False]['OID'])
    # print(len(Groundtruth_N))
    return Groundtruth_N

### NeuralCoref | M = 10, 20, 30, 50 | Select = True 

In [26]:
# rq4

Num_Src = 1000
Groundtruth_N = setup_groundtruth(cr='NeuralCoref')
N = len(Groundtruth_N)
P = Num_Src - N

select = True

for method in ('Crest',  ): # 'SIT', 'PatInv', 'CAT', 'Checklist','Textattack', 
    print(f'\nMethod = {method}')
    # df = pd.read_csv('../NewOutput_Stats/update_{}.tsv'.format(method), delimiter='\t')
    df = pd.read_csv('../Output/RQ1-IssueRevealing/generate/NeuralCoref/{}.tsv'.format(method), delimiter='\t')
    
    for m in [10, 20, 30, 50]:
        df_sampled = sample_maxGen(df, maxGen=m)
            
        if method == 'CrestNew' and select:
            df_sampled = df_sampled[df_sampled['isCorefKeep'] == True]
            
        Gen = len(df_sampled)
        Issues = len(df_sampled[df_sampled['pairConsistent'] == False])
            
        hit = 0
        for nid in Groundtruth_N:
            if False in set(df_sampled[df_sampled['OID'] == nid]['pairConsistent']) or '[]' in set(df_sampled[df_sampled['OID'] == nid]['oriCoref']) or '[]' in set(df_sampled[df_sampled['OID'] == nid]['newCoref']):
                hit += 1
        
        TP = len(df_sampled[(df_sampled['pairConsistent'] == False) & (df_sampled['isTrueBug'] == True)])
        FP = len(df_sampled[(df_sampled['pairConsistent'] == False) & (df_sampled['isTrueBug'] == False)])
        Prec = round(TP / (TP + FP), 2)
        
        if method == 'Crest' and select:
            corefChanged = len(df_sampled[(df_sampled['isCorefKeep'] == False) &  (df_sampled['depthConsistent']==True) & (df_sampled['oriDepth'] != 'set()') & (df_sampled['newDepth'] != 'set()')])
            corefKeep = Gen - corefChanged
        else:
            corefKeep = len(df_sampled[df_sampled['isCorefKeep'] == True])
            
        corefKeepRatio = round(corefKeep / Gen, 4)

        print(f'M: {m} | Select: {select} | Gen: {Gen} | Issues: {Issues} | Hit: {hit} / {N} ({round(hit/N * 100, 2)}) | TP: {TP} | FP: {FP} | Prec: {Prec} | CorefKeep: {corefKeep} / {Gen} =  {corefKeepRatio}')

999
999
1000


AssertionError: 

### NeuralCoref | M = 10, 20, 30, 50 | Select = False 

In [7]:
Num_Src = 1000
Groundtruth_N = setup_groundtruth(cr='NeuralCoref')
N = len(Groundtruth_N)
P = Num_Src - N

select = False

for method in ( 'CrestNew',  ): #  'SIT', 'PatInv', 'CAT', 'Checklist','Textattack',
    print(f'\nMethod = {method}')
    df = pd.read_csv('../NewOutput_NeuralCoref/{}.tsv'.format(method), delimiter='\t')
    
    for m in [10, 20, 30, 50]:
        df_sampled = sample_maxGen(df, maxGen=m)
            
        if method == 'CrestNew' and select:
            df_sampled = df_sampled[df_sampled['isCorefKeep'] == True]
            
        Gen = len(df_sampled)
        Issues = len(df_sampled[df_sampled['pairConsistent'] == False])
            
        hit = 0
        for nid in Groundtruth_N:
            if False in set(df_sampled[df_sampled['OID'] == nid]['pairConsistent']) or '[]' in set(df_sampled[df_sampled['OID'] == nid]['oriCoref']) or '[]' in set(df_sampled[df_sampled['OID'] == nid]['newCoref']):
                hit += 1
            
        TP = len(df_sampled[(df_sampled['pairConsistent'] == False) & (df_sampled['isTrueBug'] == True)])
        FP = len(df_sampled[(df_sampled['pairConsistent'] == False) & (df_sampled['isTrueBug'] == False)])
        Prec = round(TP / (TP + FP), 2)
        
        if method == 'CrestNew' and select:
            corefChanged = len(df_sampled[(df_sampled['isCorefKeep'] == False) &  (df_sampled['depthConsistent']==True) & (df_sampled['oriDepth'] != 'set()') & (df_sampled['newDepth'] != 'set()')])
            corefKeep = Gen - corefChanged
        else:
            corefKeep = len(df_sampled[df_sampled['isCorefKeep'] == True])
            
        corefKeepRatio = round(corefKeep / Gen, 4)

        print(f'M: {m} | Select: {select} | Gen: {Gen} | Issues: {Issues} | Hit: {hit} / {N} ({round(hit/N * 100, 2)}) | TP: {TP} | FP: {FP} | Prec: {Prec} | CorefKeep: {corefKeep} / {Gen} =  {corefKeepRatio}')

NameError: name 'pd' is not defined

### StatsCR | M = 10, 20, 30, 50 | Select = True 

In [ ]:
Num_Src = 1000
Groundtruth_N = setup_groundtruth(cr='Stats')
N = len(Groundtruth_N)
P = Num_Src - N

select = True

for method in ('SIT', 'PatInv', 'CAT', 'Checklist','Textattack', 'CrestNew',  ): #  , 'SIT', 
    print(f'\nMethod = {method}')
    # df = pd.read_csv('../NewOutput_Stats/update_{}.tsv'.format(method), delimiter='\t')
    df = pd.read_csv('../NewOutput_NeuralCoref/{}.tsv'.format(method), delimiter='\t')
    
    for m in [1, 10, 20, 30, 50]:
        df_sampled = sample_maxGen(df, maxGen=m)
            
        Gen = len(df_sampled)
        Issues = len(df_sampled[df_sampled['pairConsistent'] == False])
            
        hit = 0
        for nid in Groundtruth_N:
            if False in set(df_sampled[df_sampled['OID'] == nid]['pairConsistent']) or '[]' in set(df_sampled[df_sampled['OID'] == nid]['oriCoref']) or '[]' in set(df_sampled[df_sampled['OID'] == nid]['newCoref']):
                hit += 1
        
        if method == 'CrestNew' and select:
            df_sampled = df_sampled[df_sampled['isCorefKeep'] == True]
            
            
        TP = len(df_sampled[(df_sampled['pairConsistent'] == False) & (df_sampled['isTrueBug'] == True)])
        FP = len(df_sampled[(df_sampled['pairConsistent'] == False) & (df_sampled['isTrueBug'] == False)])
        Prec = round(TP / (TP + FP), 2)
        
        if method == 'Crest' and select:
            corefChanged = len(df_sampled[(df_sampled['isCorefKeep'] == False) &  (df_sampled['depthConsistent']==True) & (df_sampled['oriDepth'] != 'set()') & (df_sampled['newDepth'] != 'set()')])
            corefKeep = Gen - corefChanged
        else:
            corefKeep = len(df_sampled[df_sampled['isCorefKeep'] == True])
            
        corefKeepRatio = round(corefKeep / Gen, 2)

        print(f'M: {m} | Select: {select} | Gen: {Gen} | Issues: {Issues} | Hit: {hit} / {N} ({round(hit/N * 100, 2)}) | TP: {TP} | FP: {FP} | Prec: {Prec} | CorefKeep: {corefKeep} / {Gen} =  {corefKeepRatio}')

### StatsCR | M = 10, 20, 30, 50 | Select = False 

Num_Src = 1000
Groundtruth_N = setup_groundtruth(cr='Stats')
N = len(Groundtruth_N)
P = Num_Src - N

select = False

for method in ('SIT', 'PatInv', 'CAT', 'Checklist','Textattack', 'CrestNew',  ): #  , 'SIT', 
    print(f'\nMethod = {method}')
    # df = pd.read_csv('../NewOutput_Stats/update_{}.tsv'.format(method), delimiter='\t')
    df = pd.read_csv('../NewOutput_NeuralCoref/{}.tsv'.format(method), delimiter='\t')
    
    for m in [1, 10, 20, 30, 50]:
        df_sampled = sample_maxGen(df, maxGen=m)
            
        Gen = len(df_sampled)
        Issues = len(df_sampled[df_sampled['pairConsistent'] == False])
            
        hit = 0
        for nid in Groundtruth_N:
            if False in set(df_sampled[df_sampled['OID'] == nid]['pairConsistent']) or '[]' in set(df_sampled[df_sampled['OID'] == nid]['oriCoref']) or '[]' in set(df_sampled[df_sampled['OID'] == nid]['newCoref']):
                hit += 1
        
        if method == 'CrestNew' and select:
            df_sampled = df_sampled[df_sampled['isCorefKeep'] == True]
            
            
        TP = len(df_sampled[(df_sampled['pairConsistent'] == False) & (df_sampled['isTrueBug'] == True)])
        FP = len(df_sampled[(df_sampled['pairConsistent'] == False) & (df_sampled['isTrueBug'] == False)])
        Prec = round(TP / (TP + FP), 2)
        
        if method == 'Crest' and select:
            corefChanged = len(df_sampled[(df_sampled['isCorefKeep'] == False) &  (df_sampled['depthConsistent']==True) & (df_sampled['oriDepth'] != 'set()') & (df_sampled['newDepth'] != 'set()')])
            corefKeep = Gen - corefChanged
        else:
            corefKeep = len(df_sampled[df_sampled['isCorefKeep'] == True])
            
        corefKeepRatio = round(corefKeep / Gen, 2)

        print(f'M: {m} | Select: {select} | Gen: {Gen} | Issues: {Issues} | Hit: {hit} / {N} ({round(hit/N * 100, 2)}) | TP: {TP} | FP: {FP} | Prec: {Prec} | CorefKeep: {corefKeep} / {Gen} =  {corefKeepRatio}')